In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA, KernelPCA
from sklearn.neighbors import KNeighborsClassifier
import optuna
import optuna.visualization as vis

n = 12

X = np.load('Datasets/kryptonite-%s-X.npy'%(n))
y = np.load('Datasets/kryptonite-%s-y.npy'%(n))

def objective(trial):
    ### ========== Shuffle and Split Data ========== ###

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.8, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Suggest dimensionality reduction and kernel parameters
    use_pca = trial.suggest_categorical('use_pca', ['True', 'False'])
    
    

    if use_pca:
        kernel_pca = trial.suggest_categorical('pca_kernel', ['linear', 'poly', 'rbf', 'sigmoid', 'cosine'])

        params_pca = {}

        if kernel_pca == 'poly':
            params_pca['degree'] = trial.suggest_int('degree_pca', 1, 10)
        if kernel_pca in ['poly', 'rbf', 'sigmoid']:
            params_pca['gamma'] = trial.suggest_float('gamma_pca', 0.1, 10)

        params_pca['n_jobs'] = -1
        params_pca['n_components'] = trial.suggest_int('n_pca_components', 1, max(1, X_train_scaled.shape[1]))


    ### ========== Dimensionality Reduction ========== ###

    if use_pca == 'True':
        pca = KernelPCA(
            ** params_pca
        )

        X_train_scaled = pca.fit_transform(X_train_scaled)
        X_val_scaled = pca.transform(X_val)

    # KNN Classifier parameters
    classifier = KNeighborsClassifier(
        n_neighbors=trial.suggest_int('n_neighbors', 5, 50),
        weights=trial.suggest_categorical('weights', ['uniform', 'distance']),
        n_jobs=-1
    )

    # Train the classifier
    classifier.fit(X_train_scaled, y_train)

    # Evaluate on the val set
    y_val_pred = classifier.predict(X_val_scaled)
    val_accuracy = accuracy_score(y_val, y_val_pred)

    # Report validation accuracy for pruning
    trial.report(val_accuracy, step=0)

    # Check if the trial should be pruned
    if trial.should_prune():
        raise optuna.TrialPruned()


    return val_accuracy

# Create a study object with a pruner
pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)

# Optimize the objective function
study.optimize(objective, n_trials=100)

print("Best hyperparameters:", study.best_params)
print("Best value:", study.best_value)

# Visualize the optimization history and parameter importances
vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()
vis.plot_slice(study).show()

[I 2024-11-11 19:40:38,176] A new study created in memory with name: no-name-bfc86c58-a5cf-4a55-ad84-93c5129147cd
[I 2024-11-11 19:40:38,454] Trial 0 finished with value: 0.7525694444444444 and parameters: {'use_pca': 'False', 'pca_kernel': 'poly', 'degree_pca': 1, 'gamma_pca': 1.7691115740044365, 'n_pca_components': 9, 'n_neighbors': 36, 'weights': 'distance'}. Best is trial 0 with value: 0.7525694444444444.
[I 2024-11-11 19:40:38,952] Trial 1 finished with value: 0.18958333333333333 and parameters: {'use_pca': 'False', 'pca_kernel': 'linear', 'n_pca_components': 5, 'n_neighbors': 25, 'weights': 'uniform'}. Best is trial 0 with value: 0.7525694444444444.
[I 2024-11-11 19:40:39,216] Trial 2 finished with value: 0.7483333333333333 and parameters: {'use_pca': 'False', 'pca_kernel': 'rbf', 'gamma_pca': 9.211253934780128, 'n_pca_components': 6, 'n_neighbors': 32, 'weights': 'distance'}. Best is trial 0 with value: 0.7525694444444444.
[I 2024-11-11 19:40:40,187] Trial 3 finished with value:

Best hyperparameters: {'use_pca': 'False', 'pca_kernel': 'linear', 'n_pca_components': 1, 'n_neighbors': 5, 'weights': 'distance'}
Best value: 0.9518055555555556
